
# 1. Gravação de Áudio Com Python (e Uma Pitada de JavaScript) 🎤

In [32]:
# all imports
from IPython.display import Javascript, Audio, display
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))
  js_result = output.eval_js('record(%s)' % (sec * 1000))
  audio = b64decode(js_result.split(',')[1])
  file_name = 'request_audio.wav'
  with open(file_name, 'wb') as f:
    f.write(audio)
  return f'/content/{file_name}'

print("Ouvindo...\n")
record_file = record()
display(Audio(record_file, autoplay=True))

Ouvindo...



<IPython.core.display.Javascript object>

# 2. Reconhecimento de Fala com Whisper (OpenAI) 🧠


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 6.2 MB/s eta 0:00:00


In [33]:
import whisper

model = whisper.load_model("small")
result = model.transcribe(record_file, fp16=False, language='pt')
transcription = result["text"]
print(transcription)

 Aí, minha assistente de voz, beleza?


# 3. Integração com a API do ChatGPT 💬


In [27]:
!pip install -q -U google-generativeai

In [34]:
import google.generativeai as genai

# Sua chave que já está funcionando
genai.configure(api_key="")   # Coloque sua chave aqui para rodar

# Vamos descobrir quais modelos você tem acesso
modelos_disponiveis = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]

if modelos_disponiveis:
    # Escolhemos o primeiro da lista (geralmente o gemini-1.5-flash ou pro)
    modelo_nome = modelos_disponiveis[0]
    print(f"Sucesso! Usando o modelo: {modelo_nome}")
    model = genai.GenerativeModel(modelo_nome)
else:
    print("Nenhum modelo encontrado. Verifique sua chave de API.")

def gerar_fala_ia(texto_entrada):
    try:
        # Garante que temos texto para processar
        if not texto_entrada:
            return "O áudio veio vazio, tente gravar novamente."

        response = model.generate_content(texto_entrada)
        return response.text
    except Exception as e:
        return f"Erro na chamada: {e}"

# O nome da variável PRECISA ser chatgpt_response para não quebrar sua Seção 4
chatgpt_response = gerar_fala_ia(transcription)
print(f"\nIA Respondeu: {chatgpt_response}")

Sucesso! Usando o modelo: models/gemini-2.5-flash

IA Respondeu: E aí! Por aqui tá tudo beleza, pronta para o que precisar! 😄

Como posso te ajudar hoje?


# 4. Sintetizando a Resposta do ChatGPT Como Voz (gTTS) 🔊

In [14]:
!pip install gTTS

In [36]:
from gtts import gTTS
from IPython.display import Audio

# Esta linha remove os asteriscos para a IA não falar "asterisco"
texto_para_falar = chatgpt_response.replace('*', '')

# Agora passamos o texto limpo para o gTTS
tts = gTTS(text=texto_para_falar, lang='pt', slow=False)
tts.save('resposta_final.wav')

# Play no áudio
display(Audio('resposta_final.wav', autoplay=True))